## 잠재요인 협업 필터링(Latent Factor Collaborative Filtering)

### 데이터: https://grouplens.org/datasets/movielens/latest/

In [2]:
import pandas as pd
import numpy as np

In [1]:
import numpy as np
from sklearn.metrics import mean_squared_error
 
def get_rmse(R, P, Q, non_zeros):
    error = 0
    # 두개의 분해된 행렬 P와 Q.T의 내적 곱으로 예측 R 행렬 생성
    full_pred_matrix = np.dot(P, Q.T)
     
    # 실제 R 행렬에서 널이 아닌 값의 위치 인덱스 추출하여 실제 R 행렬과 예측 행렬의 RMSE 추출
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]
     
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]
       
    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)
     
    return rmse
 
 
def matrix_factorization(R, K, steps=200, learning_rate=0.01, r_lambda = 0.01):
    num_users, num_items = R.shape
    # P와 Q 매트릭스의 크기를 지정하고 정규분포를 가진 랜덤한 값으로 입력합니다.
    np.random.seed(1)
    P = np.random.normal(scale=1./K, size=(num_users, K))
    Q = np.random.normal(scale=1./K, size=(num_items, K))
 
    break_count = 0
        
    # R > 0 인 행 위치, 열 위치, 값을 non_zeros 리스트 객체에 저장.
    non_zeros = [ (i, j, R[i,j]) for i in range(num_users) for j in range(num_items) if R[i,j] > 0 ]
     
    # SGD기법으로 P와 Q 매트릭스를 계속 업데이트.
    for step in range(steps):
        for i, j, r in non_zeros:
            # 실제 값과 예측 값의 차이인 오류 값 구함
            eij = r - np.dot(P[i, :], Q[j, :].T)
            # Regularization을 반영한 SGD 업데이트 공식 적용
            P[i,:] = P[i,:] + learning_rate*(eij * Q[j, :] - r_lambda*P[i,:])
            Q[j,:] = Q[j,:] + learning_rate*(eij * P[i, :] - r_lambda*Q[j,:])
         
        rmse = get_rmse(R, P, Q, non_zeros)
        if (step % 10) == 0 :
            print("### iteration step : ", step," rmse : ", rmse)
             
    return P, Q
    
# if __name__ == "__main__":
import pandas as pd
import numpy as np

movies = pd.read_csv('./movies.csv')
ratings = pd.read_csv('./ratings.csv')
ratings = ratings[['userId', 'movieId', 'rating']]
ratings_matrix = ratings.pivot_table('rating', index='userId', columns='movieId')

# title 컬럼을 얻기 이해 movies 와 조인 수행
rating_movies = pd.merge(ratings, movies, on='movieId')

# columns='title' 로 title 컬럼으로 pivot 수행.
ratings_matrix = rating_movies.pivot_table('rating', index='userId', columns='title')


P, Q = matrix_factorization(ratings_matrix.values, K=50, steps=200, learning_rate=0.01, r_lambda = 0.01)
pred_matrix = np.dot(P, Q.T)

### iteration step :  0  rmse :  2.9023619751336867


In [1]:
import pandas as pd
import numpy as np

In [2]:
movies = pd.read_csv('./movies.csv')
ratings = pd.read_csv('./ratings.csv')
ratings = ratings[['userId', 'movieId', 'rating']]
ratings_matrix = ratings.pivot_table('rating', index='userId', columns='movieId')


In [3]:
# R 행렬과 예측 행렬의 오차를 구하는 get_rmse 함수
from sklearn.metrics import mean_squared_error

def get_rmse(R, P, Q, non_zeros):
    error = 0
    # 두 개의 분해된 행렬 P, Q.T의 내적으로 예측 R 행렬 생성
    full_pred_matrix = np.dot(P, Q.T)
    
    # 실제 R 행렬에서 null이 아닌 값의 위치 인덱스를 추출해 실제 R 행렬과 예측 행렬의 rmse 추출
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]
    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)
    
    return rmse

In [4]:
# 확률적 경사 하강법을 이용한 행렬 분해 로직 함수

def matrix_factorization(R, K, steps=200, learning_rate=0.01, r_lambda=0.01):
    num_users, num_items = R.shape
    
    # P, Q 매트릭스의 크기를 지정하고, 정규 분포를 가진 랜덤한 값으로 입력
    np.random.seed(1)
    P = np.random.normal(scale=1./K, size=(num_users, K))
    Q = np.random.normal(scale=1./K, size=(num_items, K))
            
    # R > 0 인 행 위치, 열 위치 값을 non_zeros 리스트로 저장
    non_zeros = [ (i, j, R[i, j]) for i in range(num_users) for j in range(num_items) if R[i, j] > 0 ]

    # SGD기법으로 P, Q 매트릭스를 계속 업데이트
    for step in range(steps):
        for i, j, r in non_zeros:
            # 실제 값과 예측 값의 차이(오류)
            eij = r - np.dot(P[i, :], Q[j, :].T)
            # regularization을 반영한 SGD 업데이트
            P[i, :] = P[i, :] + learning_rate*(eij * Q[j, :] - r_lambda*P[i, :])
            Q[j, :] = Q[j, :] + learning_rate*(eij * P[i, :] - r_lambda*Q[j, :])

            rmse = get_rmse(R, P, Q, non_zeros)
            if (step % 10) == 0 :
                print("iteration step:", step, "rmse: ", rmse)

    return P, Q


In [7]:
# 행렬분해
P, Q = matrix_factorization(ratings_matrix.values, K=50, steps=200, learning_rate=0.01, r_lambda=0.01)
pred_matrix = np.dot(P, Q.T)

iteration step: 0 rmse:  3.6534450570470276
iteration step: 0 rmse:  3.6534450505547125
iteration step: 0 rmse:  3.6534450281797435
iteration step: 0 rmse:  3.6534449910727407
iteration step: 0 rmse:  3.6534449644342355
iteration step: 0 rmse:  3.653444939437006
iteration step: 0 rmse:  3.653444883897086
iteration step: 0 rmse:  3.6534448081697266
iteration step: 0 rmse:  3.6534448052816155
iteration step: 0 rmse:  3.653444767610346
iteration step: 0 rmse:  3.6534447149543876
iteration step: 0 rmse:  3.6534446885259446
iteration step: 0 rmse:  3.653444666415111
iteration step: 0 rmse:  3.6534446209137865
iteration step: 0 rmse:  3.6534446541984247
iteration step: 0 rmse:  3.653444608540013
iteration step: 0 rmse:  3.6534445638001127
iteration step: 0 rmse:  3.6534445592414646
iteration step: 0 rmse:  3.653444494415644
iteration step: 0 rmse:  3.6534444588989445
iteration step: 0 rmse:  3.6534444197867693
iteration step: 0 rmse:  3.653444368470718
iteration step: 0 rmse:  3.653444357008

In [24]:
ratings_pred_matrix = pd.DataFrame(data=pred_matrix, index=ratings_matrix.index, columns=ratings_matrix.columns)
ratings_pred_matrix.head()

NameError: name 'pred_matrix' is not defined

In [ ]:
def get_unseen_movies(ratings_matrix, userId):
    # userId로 입력받은 사용자의 모든 영화 정보를 추출해 Series로 반환
    user_rating = ratings_matrix.loc[userId, :] # title을 인덱스로 가짐
    already_seen = user_rating[user_rating > 0].index.tolist() # 0보다 크면 관람했던 영화    
    movies_list = ratings_matrix.columns.tolist() # 모든 영화명을 list로
    unseen_list = [movie for movie in movies_list if movie not in already_seen] # already_seen은 movie_list에서 제외

    return unseen_list

In [ ]:
##### userId #####
n = 9
##################

# 사용자id, 추천 후보 영화 리스트, 추천 상위 영화 개수를 받아, 가장 높은 예측 평점의 영화를 사용자에게 추천
def recomm_movie_by_userid(pred_df, userId, unseen_list, top_n=10):
    # 예측 평점 DF에서 사용자id와 unseen_list의 영화명 컬럼을 추출하여, 예측 평점이 가장 높은 순으로 정렬
    recomm_movies = pred_df.loc[userId, unseen_list].sort_values(ascending=False)[:top_n]
    return recomm_movies

In [ ]:
unseen_list = get_unseen_movies(ratings_matrix, n)
recomm_movies = recomm_movie_by_userid(ratings_pred_matrix, 9, unseen_list, top_n=10)
recomm_movies = pd.DataFrame(data=recomm_movies.values, index=recomm_movies.index, columns=['pred_score'])

recomm_movies

In [6]:
import pandas as pd
import numpy as np

In [1]:
import pandas as pd
import numpy as np

# R 행렬과 예측 행렬의 오차를 구하는 get_rmse 함수
from sklearn.metrics import mean_squared_error

def get_rmse(R, P, Q, non_zeros):
    error = 0
    # 두 개의 분해된 행렬 P, Q.T의 내적으로 예측 R 행렬 생성
    full_pred_matrix = np.dot(P, Q.T)
    
    # 실제 R 행렬에서 null이 아닌 값의 위치 인덱스를 추출해 실제 R 행렬과 예측 행렬의 rmse 추출
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]

    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]
    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)
    
    return rmse

# 확률적 경사 하강법을 이용한 행렬 분해 로직 함수

def matrix_factorization(R, K, steps=200, learning_rate=0.01, r_lambda=0.01):
    num_users, num_items = R.shape
    
    # P, Q 매트릭스의 크기를 지정하고, 정규 분포를 가진 랜덤한 값으로 입력
    np.random.seed(1)
    P = np.random.normal(scale=1./K, size=(num_users, K))
    Q = np.random.normal(scale=1./K, size=(num_items, K))
    
    # R > 0 인 행 위치, 열 위치 값을 non_zeros 리스트로 저장
    non_zeros = [ (i, j, R[i, j]) for i in range(num_users) for j in range(num_items) if R[i, j] > 0 ]
    
    # SGD기법으로 P, Q 매트릭스를 계속 업데이트
    for step in range(steps):
        for i, j, r in non_zeros:
            # 실제 값과 예측 값의 차이(오류)
            eij = r - np.dot(P[i, :], Q[j, :].T)
            # regularization을 반영한 SGD 업데이트
            P[i, :] = P[i, :] + learning_rate*(eij * Q[j, :] - r_lambda*P[i, :])
            Q[j, :] = Q[j, :] + learning_rate*(eij * P[i, :] - r_lambda*Q[j, :])
            
            rmse = get_rmse(R, P, Q, non_zeros)
        if (step % 10) == 0 :
            print("iteration step:", step, "rmse: ", rmse)
    return P, Q

movies = pd.read_csv('./movies.csv')
ratings = pd.read_csv('./ratings.csv')
ratings = ratings[['userId', 'movieId', 'rating']]
ratings_matrix = ratings.pivot_table('rating', index='userId', columns='movieId')
rating_movies = pd.merge(ratings, movies, on='movieId')
ratings_matrix = rating_movies.pivot_table('rating', index='userId', columns='title')


# 행렬분해
P, Q = matrix_factorization(ratings_matrix.values, K=50, steps=200, learning_rate=0.01, r_lambda=0.01)
pred_matrix = np.dot(P, Q.T)

iteration step: 0 rmse:  3.653457190596704
iteration step: 0 rmse:  3.653457152752282
iteration step: 0 rmse:  3.653457125536853
iteration step: 0 rmse:  3.653457077902461
iteration step: 0 rmse:  3.6534570363951278
iteration step: 0 rmse:  3.6534570085398768
iteration step: 0 rmse:  3.6534569970149575
iteration step: 0 rmse:  3.653456967000257
iteration step: 0 rmse:  3.6534569426428147
iteration step: 0 rmse:  3.6534569206178404
iteration step: 0 rmse:  3.653456867037414
iteration step: 0 rmse:  3.653456787685173
iteration step: 0 rmse:  3.6534568067711084
iteration step: 0 rmse:  3.6534567849372763
iteration step: 0 rmse:  3.6534567725245615
iteration step: 0 rmse:  3.653456780726456
iteration step: 0 rmse:  3.6534567720843922
iteration step: 0 rmse:  3.6534567728192875
iteration step: 0 rmse:  3.6534566471107213
iteration step: 0 rmse:  3.653456595524333
iteration step: 0 rmse:  3.6534565887818027
iteration step: 0 rmse:  3.653456577351179
iteration step: 0 rmse:  3.653456554239872